# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [34]:
# Load the dataset
# Preprocess as per the instructions above | We have already done in TASK 2

import pandas as pd 
import numpy as np
import warnings

# Load the dataset
df = pd.read_csv('Online Retail.csv', encoding='ISO-8859-1')

# Preprocess the dataset

df.dropna(subset=['CustomerID'], inplace=True)  # Drop rows with missing Customer

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])  # Convert InvoiceDate to datetime

df['TotalPrice'] = df['Quantity'] * df['UnitPrice']  # TotalPrice column

df = df[df['Quantity'] > 0]  # Filter out negative quantities

df = df[df['UnitPrice'] > 0]  # Filter out negative unit prices

df = df[df['Country'] == 'United Kingdom']  # Filter for UK customers

df = df[df['Description'].notna()]  # Filter out rows with missing descriptions

df['Description'] = df['Description'].str.strip()  # Strip whitespace from descriptions

df = df[df['Description'] != '']  # Remove empty descriptions

df['InvoiceNo'] = df['InvoiceNo'].astype(str)  # Ensure InvoiceNo is string type

# Remove duplicates
df.drop_duplicates(subset=['InvoiceNo', 'StockCode', 'Description', 'Quantity','InvoiceDate', 'UnitPrice', 'CustomerID'], inplace=True)

# for Basket
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [35]:
from itertools import combinations


def get_frequent_itemsets(transactions, min_support):
    frequent_itemsets = []
    k = 1

    if 0 < min_support < 1:
        min_support = min_support * len(transactions)

    candidates = [(item,) for item in transactions.columns]

    while candidates:
        if k == 1:
            candidate_itemsets = candidates
        else:
            candidate_itemsets = generate_candidates(candidates, k)
        
        support_counts = calculate_support(transactions, candidate_itemsets)
        
        frequent_itemsets_k = {itemset: count for itemset, count in support_counts.items() if count >= min_support}
        
        if not frequent_itemsets_k:
            break

        frequent_itemsets.extend(frequent_itemsets_k.items())
        candidates = list(frequent_itemsets_k.keys())
        k += 1

    return frequent_itemsets

def generate_candidates(prev_frequent_itemsets, k):
    candidates = []
    for i in range(len(prev_frequent_itemsets)):
        for j in range(i + 1, len(prev_frequent_itemsets)):
            candidate = tuple(sorted(set(prev_frequent_itemsets[i]) | set(prev_frequent_itemsets[j])))
            if len(candidate) == k and candidate not in candidates:
                candidates.append(candidate)
    return candidates

def calculate_support(transactions, candidates):
    support_counts = {}
    for candidate in candidates:
        count = transactions[list(candidate)].all(axis=1).sum()
        support_counts[candidate] = count
    return support_counts

## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

In [36]:
from itertools import combinations

def generate_rules(frequent_itemsets, transactions, min_confidence):
    rules = []
    num_transactions = len(transactions)
    
    # Convert one-hot encoded DataFrame to list of sets (item names)
    transaction_sets = []
    for idx, row in transactions.iterrows():
        items = set(row.index[row == 1].tolist())
        transaction_sets.append(items)

    for itemset, support_count in frequent_itemsets:
        itemset = set(itemset)
        
        if len(itemset) < 2:
            continue
        
        subsets = [set(sub) for i in range(1, len(itemset)) for sub in combinations(itemset, i)]
        
        for subset in subsets:
            subset_support_count = sum(1 for transaction in transaction_sets if subset.issubset(transaction))
            confidence = support_count / subset_support_count if subset_support_count > 0 else 0
            
            if confidence >= min_confidence:
                rules.append((subset, itemset - subset, confidence))
    
    return rules

## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**

In [37]:
# Output the final results
# Optional: Add visualizations
min_support = 0.02     # absolute count threshold
min_confidence = 0.6  # relative threshold
frequent_itemsets = get_frequent_itemsets(basket, min_support)
rules = generate_rules(frequent_itemsets, basket, min_confidence)

In [38]:
# Print top 10 frequent itemsets
# Print top 10 association rules (by confidence or lift)
print("Top 10 Frequent Itemsets:")
for itemset, count in frequent_itemsets[:10]:
    print(f"Itemset: {itemset}, Support: {count}")

print("\nTop 10 Association Rules:")
for antecedent, consequent, confidence in rules[:10]:
    print(f"Rule: {set(antecedent)} -> {set(consequent)}, Confidence: {confidence:.2f}")
    
    

Top 10 Frequent Itemsets:
Itemset: ('3 STRIPEY MICE FELTCRAFT',), Support: 373
Itemset: ('6 RIBBONS RUSTIC CHARM',), Support: 628
Itemset: ('60 CAKE CASES VINTAGE CHRISTMAS',), Support: 429
Itemset: ('60 TEATIME FAIRY CAKE CASES',), Support: 587
Itemset: ('72 SWEETHEART FAIRY CAKE CASES',), Support: 444
Itemset: ('ALARM CLOCK BAKELIKE GREEN',), Support: 690
Itemset: ('ALARM CLOCK BAKELIKE IVORY',), Support: 424
Itemset: ('ALARM CLOCK BAKELIKE PINK',), Support: 491
Itemset: ('ALARM CLOCK BAKELIKE RED',), Support: 758
Itemset: ('ANTIQUE SILVER T-LIGHT GLASS',), Support: 522

Top 10 Association Rules:
Rule: {'ALARM CLOCK BAKELIKE GREEN'} -> {'ALARM CLOCK BAKELIKE RED'}, Confidence: 0.66
Rule: {'GARDENERS KNEELING PAD CUP OF TEA'} -> {'GARDENERS KNEELING PAD KEEP CALM'}, Confidence: 0.73
Rule: {'GARDENERS KNEELING PAD KEEP CALM'} -> {'GARDENERS KNEELING PAD CUP OF TEA'}, Confidence: 0.62
Rule: {'PINK REGENCY TEACUP AND SAUCER'} -> {'GREEN REGENCY TEACUP AND SAUCER'}, Confidence: 0.82
Rule: